# 1.LLM Architecture 

1.GPT类型的大模型包括一下几个部分：
- token and positional embeddings
- dropout
- 一系列transformer blocks
- 一个layer normalization
- 一个输出的linear layer(输出最终的logits) 

Q: PDF书中的 Figure 4.4中说GPT model输出的vector的维度与input token embedding 维度是一样的，但是在 section 4.1的最后，对于输出的logits的介绍中发现，输出的logits中每个token的维度是vocab_size的大小，这就和Figure 4.4中不一致了，后续需要确认一下。

A: 这里GPT model的输出指的是堆叠的Transformer block的输出，因为transformer block不会改变input sequence的维度，因此其输出的vector的维度和输入的embedding size是一样的。但是经过linear projection之后，维度就会变到vocab_size，这是为了后续得到预测的输出token。

# 2.使用Layer Normalization归一化激活值

1.使用Layer Normalization的原因<br>
训练层数非常多的神经网络会出现梯度消失(vanishing)或爆炸(exploding)的问题，使得模型的训练会十分不稳定(unstable)，也就是参数的更新会很困难。

2.layer normalization<br>
（1）主要思想<br>
将神经网络中某一层输出的激活值调整为均值为0，方差为1（也叫做单位方差，unit variance）。<br>
（2）作用<br>
加速模型的收敛，并且保证模型训练过程的稳定。

3.layer normalization和batch normalization的区别<br>
（1）归一化的维度不同<br>
batch normalization是对batch维度进行归一化，而layer normalization是对特征维度进行归一化。

（2）使用情况不同<br>
batch normalization因为是对batch维度进行归一化，而LLM需要大量的计算资源，因此batch size通常会因为有限的计算资源而设置得非常小，进而就会影响batch normalization的效果。<br>
但是layer normalization是对特征维度进行归一化，与batch size无关，就不会受到有限计算资源的影响，更加灵活和稳定，特别适用于分布式训练或者计算资源十分有限的场景。

# 3.带GELU激活函数的前向传播网络FFN

1.LLM中常用的激活函数
除了ReLU(Rectified Linear Unit )激活函数以外，在LLM常用的还有<br>
（1）GELU(Gaussian Error Linear Unit)：高斯误差线性单元。公式为：<br>
$GELU(x) =\Phi(x)$, $\Phi$是标准高斯分布的累积分布函数。 数学近似的公式为：<br>
<div align="center">
    <img src="./images/GELU激活函数的近似公式.jpg">
</div>

优势是：<br>
在训练过程中接收negative输入的神经元对学习过程仍然有贡献，虽然贡献要比接收positive输入的神经元小得多。

（2）SwiGLU(Sigmoid-Weighted/Swish-Gated Linear Unit)：Sigmoid加权线性单元.公式为：
<div align="center" style="height:500;width:500;">
    <img src="./images/SwiGLU激活函数的数学表达式.jpg">
</div>

2.Feed Forward Network的作用
<p style="text-indent: 2em;">
加强模型从数据中学习和泛化的能力。并且允许模型探索更加丰富的表示空间，因为会先对特征进行expanding，然后再contraction。
</p>
    

# 4.shortcut connection

1.概念
<p style="text-indent: 2em">
    又叫做skip / residual connection。跳跃连接或者残差连接。最早是在计算机视觉领域中提出来-ResNet，以避免梯度消失的问题，使得模型中靠前的layers能够得到有效训练。 
</p>

2.原理和实现方式
<div style="text-indent: 2em">
    （1）原理：梯度在网络中反传的时候，通过跳过一个或者多个layers，以实现更短的传递路径。
</div>
<div style="text-indent: 2em">
    （2）实现方式：第一层的输入就是原本的输出，在此之后将每一层的输出，直接加到下一层的输入上。
</div>


# 5.transformer block

1.transformer block的结构 <br>
<div align="center" >
    <img src="./images/transformer_block结构.jpg" style="height:500px; width:500px;">
</div>

2.layer normalization的位置
在GPT的transformer block中layer normalization是放在多头注意力和FFN之前的，这叫做Pre-LayerNorm。

而在原始transformer的实现中，是将layer normalization放在多头注意力和FFN之后的，这叫做Post-LayerNorm，这种方式很容易导致训练动态性变差。

3.特点
transformer block输出的shape和输入是一样的，该特性使得它非常适用于sequence-to-sequence的任务。虽然sequence的length和feature size没变，但实际上每一个输出的vector都被重新编码了，包含了整个input sequence的上下文信息。

# 6. GPT model

1.124M的GPT-2参数问题

在GPT-2中使用了权重绑定（Weight tying）技术，具体来讲，就是将token embedding layer和最后的output head layer和最后的output layer的权重进行共享，因为这两个layer的参数矩阵shape都是(vocab_size, emb_dim)。这样的好处是减少总的内存消耗和模型的计算复杂度——但是`根据Sebastian Raschka的经验，将这两个参数矩阵分开，训练的效果会更好`。


# 7. 生成text